In [1]:
!pip install qiskit-aer

In [2]:
# check_environment.py
import importlib

def check_import(module_name):
    try:
        importlib.import_module(module_name)
        print(f"✓ {module_name} is available")
        return True
    except ImportError:
        print(f"✗ {module_name} is NOT available")
        return False

print("Checking Qiskit environment...")
modules = [
    'qiskit', 
    'qiskit_aer', 
    'qiskit.circuit.library',
    'qiskit.quantum_info',
    'qiskit.algorithms.minimum_eigensolvers',
    'qiskit.algorithms.optimizers',
    'qiskit.primitives'
]

available = [check_import(module) for module in modules]

if all(available):
    print("\nAll modules available! You can use the full VQE implementation.")
else:
    print("\nSome modules missing. Use the simplified implementation.")
!pip show qiskit

Checking Qiskit environment...
✓ qiskit is available
✓ qiskit_aer is available
✓ qiskit.circuit.library is available
✓ qiskit.quantum_info is available
✗ qiskit.algorithms.minimum_eigensolvers is NOT available
✗ qiskit.algorithms.optimizers is NOT available
✓ qiskit.primitives is available

Some modules missing. Use the simplified implementation.
Name: qiskit
Version: 1.2.4
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: C:\Users\10179\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer, qiskit-ibm-runtime


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import transpile
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

def create_ising_hamiltonian(n_qubits=6, J=1.0, h=1.0):
    """Create 6-qubit Ising model Hamiltonian"""
    operators = []
    coefficients = []
    
    # ZZ interaction terms
    for i in range(n_qubits - 1):
        op = ['I'] * n_qubits
        op[i] = 'Z'
        op[i+1] = 'Z'
        operators.append(''.join(op))
        coefficients.append(-J)
    
    # X field terms
    for i in range(n_qubits):
        op = ['I'] * n_qubits
        op[i] = 'X'
        operators.append(''.join(op))
        coefficients.append(-h)
    
    return SparsePauliOp(operators, coefficients)

def compute_exact_ground_state(hamiltonian):
    """Compute exact ground state energy and ground state"""
    H_matrix = hamiltonian.to_matrix()
    eigenvalues, eigenvectors = np.linalg.eigh(H_matrix)
    ground_state_energy = eigenvalues[0]
    ground_state = eigenvectors[:, 0]
    return ground_state_energy, ground_state

def print_state_vector_as_array(statevector, title="Quantum State Vector", threshold=0.001):
    """Print quantum state vector as array"""
    print(f"\n{title} as Array:")
    print("=" * 60)
    
    # Convert statevector to numpy array
    state_array = np.array(statevector).reshape(-1, 1)
    
    # Print array shape
    print(f"Shape: {state_array.shape}")
    
    # Print full array
    print("\nFull state vector array:")
    np.set_printoptions(precision=6, suppress=True, linewidth=120)
    print(state_array)
    
    # Show main components (probability > threshold)
    print(f"\nMain components (probability > {threshold}):")
    probabilities = np.abs(statevector)**2
    for i, (amplitude, probability) in enumerate(zip(statevector, probabilities)):
        if probability > threshold:
            state_str = format(i, '06b')
            print(f"|{state_str}>: {amplitude.real:.6f} + {amplitude.imag:.6f}j (prob: {probability:.6f})")
    
    # Verify normalization
    norm = np.sum(probabilities)
    print(f"\nNorm (should be 1.0): {norm:.10f}")
    
    return state_array

def plot_state_vector(statevector, title="Quantum State Vector", threshold=0.01, max_states=16):
    """Plot probability distribution of quantum state vector"""
    # Calculate probabilities
    probabilities = np.abs(statevector)**2
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot full probability distribution
    states = [format(i, '06b') for i in range(len(statevector))]
    ax1.bar(states, probabilities, color='skyblue', alpha=0.7)
    ax1.set_xlabel('Basis States')
    ax1.set_ylabel('Probability')
    ax1.set_title(f'{title} - Full Distribution')
    ax1.tick_params(axis='x', rotation=90)
    
    # Only show states with probability above threshold
    significant_indices = np.where(probabilities > threshold)[0]
    if len(significant_indices) > max_states:
        # If too many significant states, only show top max_states
        top_indices = np.argsort(probabilities)[-max_states:]
        significant_indices = top_indices
    
    if len(significant_indices) > 0:
        significant_states = [states[i] for i in significant_indices]
        significant_probs = [probabilities[i] for i in significant_indices]
        
        ax2.bar(significant_states, significant_probs, color='lightcoral', alpha=0.7)
        ax2.set_xlabel('Basis States')
        ax2.set_ylabel('Probability')
        ax2.set_title(f'{title} - Significant Components (p > {threshold})')
        ax2.tick_params(axis='x', rotation=45)
        
        # Add probability values on bars
        for i, v in enumerate(significant_probs):
            ax2.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def print_state_vector(statevector, title="Quantum State Vector", threshold=0.001, plot=True, print_array=True):
    """Print quantum state vector with option to plot and print as array"""
    print(f"\n{title}:")
    print("=" * 60)
    
    # Print as array
    if print_array:
        state_array = print_state_vector_as_array(statevector, title, threshold)
    
    # Plot if requested
    if plot:
        plot_state_vector(statevector, title, threshold)
    
    return state_array if print_array else None

class VQE:
    """Professional VQE implementation class"""
    
    def __init__(self, ansatz, hamiltonian, estimator_backend=None):
        self.ansatz = ansatz
        self.hamiltonian = hamiltonian
        self.backend = estimator_backend if estimator_backend else Aer.get_backend('statevector_simulator')
        self.energy_history = []
        self.state_history = []  # Store state vector history
        
    def expectation_value(self, params):
        """Calculate energy expectation value for given parameters"""
        # Bind parameters to circuit
        param_circuit = self.ansatz.assign_parameters(params)
        
        # Compile and run circuit
        compiled_circuit = transpile(param_circuit, self.backend)
        job = self.backend.run(compiled_circuit)
        result = job.result()
        
        # Get state vector
        statevector = result.get_statevector()
        
        # Store state vector
        self.state_history.append(statevector.copy())
        
        # Calculate energy expectation value
        energy = np.real(statevector.expectation_value(self.hamiltonian))
        
        return energy
    
    def cost_function(self, params):
        """Cost function - returns energy expectation value"""
        energy = self.expectation_value(params)
        self.energy_history.append(energy)
        return energy
    
    def callback(self, xk):
        """Callback function to track optimization progress"""
        current_energy = self.cost_function(xk)
        if len(self.energy_history) % 10 == 0:
            print(f"Iteration {len(self.energy_history)}: Energy = {current_energy:.6f}")

def professional_vqe():
    """Professional VQE implementation"""
    print("Professional VQE for 6-qubit Ising model")
    
    # Create Hamiltonian
    H = create_ising_hamiltonian(6, J=1.0, h=1.0)
    print(f"Created Hamiltonian with {len(H)} terms")
    
    # Compute exact ground state energy
    exact_energy, exact_state = compute_exact_ground_state(H)
    print(f"Exact ground state energy: {exact_energy:.6f}")
    
    # Display exact ground state vector and plot
    exact_state_array = print_state_vector(exact_state, "Exact Ground State Vector", plot=True, print_array=True)
    
    # Create parameterized quantum circuit
    ansatz = EfficientSU2(6, reps=3, entanglement='linear')
    n_params = ansatz.num_parameters
    print(f"\nCreated ansatz with {n_params} parameters")
    print(f"Circuit depth: {ansatz.depth()}")
    print(ansatz.decompose())
    
    # Initialize parameters
    initial_params = np.random.random(n_params) * 2 * np.pi
    
    # Create VQE instance
    backend = Aer.get_backend('statevector_simulator')
    vqe = VQE(ansatz, H, backend)
    
    # Output initial quantum state (with random parameters) and plot
    print("\n" + "="*60)
    print("INITIAL QUANTUM STATE (with random parameters)")
    print("="*60)
    initial_circuit = ansatz.assign_parameters(initial_params)
    compiled_initial = transpile(initial_circuit, backend)
    job = backend.run(compiled_initial)
    result_sv = job.result()
    initial_statevector = result_sv.get_statevector()
    initial_state_array = print_state_vector(initial_statevector, "Initial Quantum State Vector", plot=True, print_array=True)
    
    # Initial energy
    initial_energy = vqe.cost_function(initial_params)
    print(f"\nInitial energy: {initial_energy:.6f}")
    
    # Use scipy.optimize.minimize for optimization
    print("\nStarting optimization with COBYLA...")
    result = minimize(
        vqe.cost_function,
        x0=initial_params,
        method="COBYLA",
        options={"maxiter": 1000},  
        callback=vqe.callback
    )
    
    # Output optimization results
    print(f"\nOptimization completed: {result.message}")
    print(f"Number of iterations: {result.nfev}")
    print(f"Final VQE energy: {result.fun:.6f}")
    print(f"Exact ground state energy: {exact_energy:.6f}")
    print(f"Absolute error: {abs(result.fun - exact_energy):.6f}")
    print(f"Relative error: {abs((result.fun - exact_energy) / exact_energy * 100):.4f}%")
    
    # Plot energy convergence
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(vqe.energy_history, 'b-', linewidth=2, label='VQE Energy')
    plt.axhline(y=exact_energy, color='r', linestyle='--', label='Exact Energy')
    plt.xlabel('Iteration')
    plt.ylabel('Energy')
    plt.title('VQE Energy Convergence')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot error convergence
    plt.subplot(2, 1, 2)
    errors = [abs(energy - exact_energy) for energy in vqe.energy_history]
    plt.semilogy(errors, 'g-', linewidth=2)
    plt.xlabel('Iteration')
    plt.ylabel('Absolute Error')
    plt.title('VQE Error Convergence (Log Scale)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Get optimal parameter state
    optimal_circuit = ansatz.assign_parameters(result.x)
    compiled_optimal = transpile(optimal_circuit, backend)
    job = backend.run(compiled_optimal)
    result_sv = job.result()
    vqe_statevector = result_sv.get_statevector()
    
    # Display VQE ground state vector and plot
    print("\n" + "="*60)
    print("FINAL VQE GROUND STATE VECTOR")
    print("="*60)
    vqe_state_array = print_state_vector(vqe_statevector, "VQE Ground State Vector", plot=True, print_array=True)
    
    # Calculate fidelity
    fidelity = np.abs(np.dot(vqe_statevector.conj(), exact_state))**2
    print(f"\nFidelity between VQE and exact ground state: {fidelity:.6f}")
    
    # Fix energy variance calculation - convert Statevector to numpy array
    H_matrix = H.to_matrix()
    vqe_statevector_array = np.array(vqe_statevector)
    
    # Calculate energy variance <H^2> - <H>^2
    H2_expectation = np.real(np.vdot(vqe_statevector_array, H_matrix @ H_matrix @ vqe_statevector_array))
    energy_variance = H2_expectation - (result.fun)**2
    print(f"Energy variance: {energy_variance:.6f}")
    
    # Output state vector evolution during optimization
    print("\n" + "="*60)
    print("STATE VECTOR EVOLUTION DURING OPTIMIZATION")
    print("="*60)
    
    # Select key iteration points to output state vectors
    key_iterations = [0, len(vqe.state_history)//4, len(vqe.state_history)//2, 
                     3*len(vqe.state_history)//4, len(vqe.state_history)-1]
    
    # Plot state vectors at key points during optimization
    plt.figure(figsize=(15, 10))
    gs = gridspec.GridSpec(3, 2)
    
    for i, iter_idx in enumerate(key_iterations):
        if iter_idx < len(vqe.state_history):
            state = vqe.state_history[iter_idx]
            energy = vqe.energy_history[iter_idx]
            
            # Select subplot position
            if i < 3:
                ax = plt.subplot(gs[i, 0])
            else:
                ax = plt.subplot(gs[i-3, 1])
            
            # Calculate probabilities
            probabilities = np.abs(state)**2
            
            # Only show states with probability > 0.01
            significant_indices = np.where(probabilities > 0.01)[0]
            if len(significant_indices) > 0:
                significant_states = [format(j, '06b') for j in significant_indices]
                significant_probs = [probabilities[j] for j in significant_indices]
                
                bars = ax.bar(significant_states, significant_probs, color='lightgreen', alpha=0.7)
                ax.set_title(f'Iteration {iter_idx}\nEnergy: {energy:.4f}')
                ax.tick_params(axis='x', rotation=45)
                
                # Add probability values on bars
                for bar, prob in zip(bars, significant_probs):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                           f'{prob:.3f}', ha='center', va='bottom', fontsize=8)
    
    plt.suptitle('Quantum State Evolution During VQE Optimization', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    
    # Return all important state vector arrays
    return {
        'vqe_energy': result.fun,
        'optimal_params': result.x,
        'exact_energy': exact_energy,
        'fidelity': fidelity,
        'state_history': vqe.state_history,
        'exact_state_array': exact_state_array,
        'initial_state_array': initial_state_array,
        'vqe_state_array': vqe_state_array
    }

def compare_optimizers():
    """Compare performance of different optimizers"""
    print("Comparing different optimizers for VQE")
    
    # Create Hamiltonian and ansatz
    H = create_ising_hamiltonian(6)
    exact_energy, exact_state = compute_exact_ground_state(H)
    ansatz = EfficientSU2(6, reps=2, entanglement='linear')
    
    optimizers = [
        ("COBYLA", {"maxiter": 50}),  # Reduced from 100 to avoid timeout
        ("Nelder-Mead", {"maxiter": 50}),
        ("Powell", {"maxiter": 50}),
    ]
    
    results = {}
    
    for opt_name, opt_options in optimizers:
        print(f"\nTesting {opt_name} optimizer...")
        
        # Initialize parameters
        initial_params = np.random.random(ansatz.num_parameters) * 2 * np.pi
        
        # Create VQE instance
        backend = Aer.get_backend('statevector_simulator')
        vqe = VQE(ansatz, H, backend)
        
        # Run optimization
        result = minimize(
            vqe.cost_function,
            x0=initial_params,
            method=opt_name,
            options=opt_options
        )
        
        # Store results
        results[opt_name] = {
            'final_energy': result.fun,
            'error': abs(result.fun - exact_energy),
            'iterations': result.nfev,
            'success': result.success,
            'initial_state': vqe.state_history[0] if vqe.state_history else None,
            'final_state': vqe.state_history[-1] if vqe.state_history else None
        }
        
        print(f"  Final energy: {result.fun:.6f}")
        print(f"  Error: {abs(result.fun - exact_energy):.6f}")
        print(f"  Iterations: {result.nfev}")
        print(f"  Success: {result.success}")
    
    # Display comparison results
    print("\n" + "="*60)
    print("OPTIMIZER COMPARISON RESULTS")
    print("="*60)
    for opt_name, res in results.items():
        print(f"{opt_name:12} | Energy: {res['final_energy']:8.6f} | "
              f"Error: {res['error']:8.6f} | Iterations: {res['iterations']:3d} | "
              f"Success: {res['success']}")
    
    # Plot comparison of different optimizer results
    plt.figure(figsize=(12, 8))
    
    # Plot final energy comparison
    plt.subplot(2, 2, 1)
    opt_names = list(results.keys())
    final_energies = [results[opt]['final_energy'] for opt in opt_names]
    errors = [results[opt]['error'] for opt in opt_names]
    
    bars = plt.bar(opt_names, final_energies, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.axhline(y=exact_energy, color='r', linestyle='--', label='Exact Energy')
    plt.ylabel('Final Energy')
    plt.title('Final Energy Comparison')
    plt.legend()
    
    # Add values on bars
    for bar, energy in zip(bars, final_energies):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                f'{energy:.4f}', ha='center', va='bottom')
    
    # Plot error comparison
    plt.subplot(2, 2, 2)
    bars = plt.bar(opt_names, errors, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.ylabel('Absolute Error')
    plt.title('Error Comparison')
    
    # Add values on bars
    for bar, error in zip(bars, errors):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.001,
                f'{error:.4f}', ha='center', va='bottom')
    
    # Plot iteration count comparison
    plt.subplot(2, 2, 3)
    iterations = [results[opt]['iterations'] for opt in opt_names]
    bars = plt.bar(opt_names, iterations, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.ylabel('Iterations')
    plt.title('Iteration Count Comparison')
    
    # Add values on bars
    for bar, iter_count in zip(bars, iterations):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                f'{iter_count}', ha='center', va='bottom')
    
    # Plot success rate comparison
    plt.subplot(2, 2, 4)
    success_rates = [1.0 if results[opt]['success'] else 0.0 for opt in opt_names]
    bars = plt.bar(opt_names, success_rates, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.ylabel('Success (1=Yes, 0=No)')
    plt.title('Optimization Success')
    plt.ylim(0, 1.1)
    
    # Add values on bars
    for bar, success in zip(bars, success_rates):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.05,
                f'{"Yes" if success else "No"}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    return results

if __name__ == "__main__":
    # Run professional VQE
    vqe_results = professional_vqe()
    
    # Compare different optimizers
    print("\n" + "="*60)
    comparison_results = compare_optimizers()
    
    # Output key state vectors as arrays
    print("\n" + "="*60)
    print("KEY STATE VECTORS AS ARRAYS")
    print("="*60)
    
    print("\nExact ground state vector shape:", vqe_results['exact_state_array'].shape)
    print("Initial state vector shape:", vqe_results['initial_state_array'].shape)
    print("VQE ground state vector shape:", vqe_results['vqe_state_array'].shape)